RGB

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os
zip_file = "/content/drive/MyDrive/archive.zip"
extract_dir = "final_data"
if not os.path.exists(extract_dir) or not os.listdir(extract_dir):
  os.makedirs(extract_dir, exist_ok=True)
  with zipfile.ZipFile(zip_file, 'r') as ref:
    ref.extractall(extract_dir)
  print('Extraction complete')
else:
    print("Extraction skipped/already exists.")

Extraction complete


# CNN


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import tensorflow as tf
from collections import defaultdict

# Load predictions and labels
true_labels = np.load("CNNTrueLabelsRGB.npy")
cnn_preds = np.load("CNNPredLabelsRGB.npy")
patches_df = pd.read_csv("/content/patches_rgb_val_info.csv")

# Class Labels
class_names = ['bike', 'bus', 'car', 'person', 'sign', 'motor', 'light', 'truck']

# Load validation dataset (to get .file_paths)
classes = {cls: i for i, cls in enumerate(class_names)}
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/final_data/Val_RGB/val_rgb_patches",
    labels='inferred',
    label_mode='int',
    class_names=list(classes.keys()),
    image_size=(64, 64),
    batch_size=512,
    shuffle=False,
)

# Extract list of file paths
val_file_paths = list(val_dataset.file_paths)

# Find misclassified samples
misclassified_indices = np.where(true_labels != cnn_preds)[0]

# Select 2 misclassified samples from each class
grouped_misclassified = defaultdict(list)
for idx in misclassified_indices:
    true_class = true_labels[idx]
    if len(grouped_misclassified[true_class]) < 2:
        grouped_misclassified[true_class].append(idx)

# Visualize
for class_id, indices in grouped_misclassified.items():
    for idx in indices:
        patch_path = val_file_paths[idx]
        patch_filename = os.path.basename(patch_path)

        # Get original image + bbox
        row = patches_df[patches_df['patch_filename'] == patch_filename].iloc[0]
        original_img_path = os.path.join(
            '/content/final_data/Val_RGB/',
            row['original_filename']
        )
        bbox = eval(str(row['bbox']))

        # Load images
        original_img = Image.open(original_img_path)
        patch_img = Image.open(patch_path)

        # Plot side-by-side
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))

        # Original image with bbox
        ax[0].imshow(original_img)
        ax[0].set_title(f"True: {class_names[true_labels[idx]]}, Pred: {class_names[cnn_preds[idx]]}")
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='red', facecolor='none')
        ax[0].add_patch(rect)

        # Cropped patch
        ax[1].imshow(patch_img)
        ax[1].set_title("Cropped Patch")

        plt.tight_layout()
        plt.show()

Output hidden; open in https://colab.research.google.com to view.

# VGG



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import tensorflow as tf
from collections import defaultdict

# Load predictions and labels
vgg_true_labels = np.load("VGGTrueLabelsRGB.npy")
vgg_preds = np.load("VGGPredLabelsRGB.npy")
patches_df = pd.read_csv("/content/patches_rgb_val_info.csv")

# Class Labels
class_names = ['bike', 'bus', 'car', 'person', 'sign', 'motor', 'light', 'truck']

# Load validation dataset (to get .file_paths)
classes = {cls: i for i, cls in enumerate(class_names)}
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/final_data/Val_RGB/val_rgb_patches",
    labels='inferred',
    label_mode='int',
    class_names=list(classes.keys()),
    image_size=(64, 64),
    batch_size=512,
    shuffle=False,
)

# Extract list of file paths
val_file_paths = list(val_dataset.file_paths)

# Find misclassified samples
misclassified_indices_vgg = np.where(vgg_true_labels != vgg_preds)[0]

# Select 2 misclassified samples from each class
grouped_vgg_misclassified = defaultdict(list)
for idx in misclassified_indices_vgg:
    true_class = vgg_true_labels[idx]
    if len(grouped_vgg_misclassified[true_class]) < 2:
        grouped_vgg_misclassified[true_class].append(idx)

# Visualize
for class_id, indices in grouped_vgg_misclassified.items():
    for idx in indices:
        patch_path = val_file_paths[idx]
        patch_filename = os.path.basename(patch_path)

        # Get original image + bbox
        if patch_filename in patches_df['patch_filename'].values:
          row = patches_df[patches_df['patch_filename'] == patch_filename].iloc[0]
          original_img_path = os.path.join(
              '/content/final_data/Val_RGB/',
              row['original_filename']
          )
          bbox = eval(str(row['bbox']))

          # Load images
          original_img = Image.open(original_img_path)
          patch_img = Image.open(patch_path)

          # Plot side-by-side
          fig, ax = plt.subplots(1, 2, figsize=(12, 6))

          # Original image with bbox
          ax[0].imshow(original_img)
          ax[0].set_title(f"True: {class_names[vgg_true_labels[idx]]}, Pred: {class_names[vgg_preds[idx]]}")
          rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='red', facecolor='none')
          ax[0].add_patch(rect)

          # Cropped patch
          ax[1].imshow(patch_img)
          ax[1].set_title("Cropped Patch")

          plt.tight_layout()
          plt.show()
        else:
            print(f"Warning: patch_filename '{patch_filename}' not found in patches_df. Skipping visualization.")

Output hidden; open in https://colab.research.google.com to view.

# Autoencoder

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import tensorflow as tf
from collections import defaultdict

# Load predictions and labels
ae_true_labels = np.load("AETrueLabelsRGB.npy")
ae_preds = np.load("AEPredLabelsRGB.npy")
patches_df = pd.read_csv("/content/patches_rgb_val_info.csv")

# Class Labels
class_names = ['bike', 'bus', 'car', 'person', 'sign', 'motor', 'light', 'truck']

# Load validation dataset (to get .file_paths)
classes = {cls: i for i, cls in enumerate(class_names)}
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/final_data/Val_RGB/val_rgb_patches",
    labels='inferred',
    label_mode='int',
    class_names=list(classes.keys()),
    image_size=(64, 64),
    batch_size=512,
    shuffle=False,
)

# Extract list of file paths
val_file_paths = list(val_dataset.file_paths)

# Find misclassified samples
misclassified_indices_ae = np.where(ae_true_labels != ae_preds)[0]

# Select 2 misclassified samples from each class
grouped_ae_misclassified = defaultdict(list)
for idx in misclassified_indices_ae:
  true_class = ae_true_labels[idx]
  if len(grouped_ae_misclassified[true_class]) < 2:
      grouped_ae_misclassified[true_class].append(idx)

# Visualize
for class_id, indices in grouped_ae_misclassified.items():
  for idx in indices:
    if idx < len(val_file_paths):
      patch_path = val_file_paths[idx]
      patch_filename = os.path.basename(patch_path)

      # Get original image + bbox
      if patch_filename in patches_df['patch_filename'].values:
        row = patches_df[patches_df['patch_filename'] == patch_filename].iloc[0]
        original_img_path = os.path.join(
            '/content/final_data/Val_RGB/',
            row['original_filename']
        )
        bbox = eval(str(row['bbox']))

        # Load images
        original_img = Image.open(original_img_path)
        patch_img = Image.open(patch_path)

        # Plot side-by-side
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))

        # Original image with bbox
        ax[0].imshow(original_img)
        ax[0].set_title(f"True: {class_names[ae_true_labels[idx]]}, Pred: {class_names[ae_preds[idx]]}")
        rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='red', facecolor='none')
        ax[0].add_patch(rect)

        # Cropped patch
        ax[1].imshow(patch_img)
        ax[1].set_title("Cropped Patch")

        plt.tight_layout()
        plt.show()
      else:
          print(f"Warning: patch_filename '{patch_filename}' not found in patches_df. Skipping visualization.")
    else:
      print(f"Warning: Index {idx} is out of bounds for val_file_paths. Skipping visualization.")

Output hidden; open in https://colab.research.google.com to view.